# Tutorial 1: Pipeline Equation Derivation

## 1. Derivation of the pipeline flow equation

There are several gas pipeline equations can be used to calculate the pipeline steady-state flow rate. The common variants are mostly originates from the 1st order Euler isothermal equations, where the first equation represents the mass conservation and the second on the energy.

$$
\frac{\partial\rho}{\partial t}+\frac{\partial(\rho v)}{\partial x} = 0,\\ \frac{\partial(\rho v)}{\partial t}+\frac{\partial(\rho v^2)}{\partial x}+\frac{\partial p}{\partial x}+f \frac{v|v|}{2D}\rho+\rho g \sin\theta=0
$$

In the static case, the velocity of the gas flow $v$ keeps constant along the pipeline and the density $\rho$ does not vary during time. Therefore, the first equation and the first two terms in the second equation can be neglected. So now we have:

$$
\frac{\partial p}{\partial x} + \frac{f\rho}{2D}|v|v + \rho g\sin(\theta) = 0
$$

Now let's assume that the volumetric gas flow rate $Q = A\cdot v$, where $A$ is the cross-section area of the pipe. And $\rho = \frac{MP}{ZRT}$, where $M=dM_{air}$ is the molar mass of gas ($d$ is the relative density and $M_{air}$ is the air molar mass which is around 28.96 g/mol), $P$ is the pressure, $Z$ is the gas compressibility factor and $R $ is the gas constant (8314.41 J/(kmol K)), $T$ is the temperature in K. So we get:

$$
\frac{\partial P}{\partial x} + \frac{fMP}{2ZRTDA^2}|Q|Q + g\sin(\theta)\frac{PM}{ZRT} = 0
$$

Because now $P$ is only dependent on $x$, so we change the partial derivative symbol to $\mathrm{d}$.

$$
\mathrm{d}P = -\frac{fMP}{2ZRTDA^2}|Q|Q\mathrm{d}x - g\sin(\theta)\frac{PM}{ZRT}\mathrm{d}x
$$

Multiplying both sides with $\rho$ ($\rho=\frac{MP}{ZRT}$)

$$
\frac{MP}{ZRT}\mathrm{d}P = -\frac{fM^2P^2}{2Z^2R^2T^2DA^2}|Q|Q\mathrm{d}x - g\sin(\theta)\frac{P^2M^2}{Z^2R^2T^2}\mathrm{d}x
$$

Now the only variables we have in the equation are the pressure $P$ and the volumetric flow rate $Q$, which are the ones we use to model our gas flow simulation. Now we calculate the integral of the above equation and we can get:

$$
\frac{M}{ZRT}\frac{P_2^2-P_1^2}{2} = -\frac{fM^2P^2}{2Z^2R^2T^2DA^2}|Q|QL - g\sin(\theta)\frac{P^2M^2}{Z^2R^2T^2}L
$$

Because we have replaced $\rho$ with $\frac{MP}{ZRT}$ and the density of the gas mixture depends on the pressure and temperature of the gas mixture. And the compressibility factor $Z$ also depends on the gas temperature and pressure. Therefore we assume there exists an average value for the pressure, temperature and compressibility factor of the transported gas mixture.

$$
\frac{M}{Z_{avg}RT_{avg}}\frac{P_2^2-P_1^2}{2} = -\frac{fM^2P_{st}^2}{2Z_{st}^2R^2T_{st}^2DA^2}|Q_{st}|Q_{st}L - g\sin(\theta)\frac{P_{avg}^2M^2}{Z_{avg}^2R^2T_{avg}^2}L
$$

By some further simplifications we can get the flow equation we need for our steady-state gas flow simulation.
$$
Q = \pm\pi\sqrt{\frac{R}{16M_{air}}}\frac{T_{st}}{P_{st}}D^{2.5}\sqrt{\frac{|P_1^2-P_2^2-2g\sin\theta L\frac{P_{avg}^2dM_{air}}{Z_{avg}RT_{avg}}|}{LdTZ_{avg}f}}
$$

## 2. Translate the pipeline equation into code

### Import dependencies and assign constants

In [3]:
import numpy as np
from scipy.constants import bar, atm, zero_Celsius, pi, R, g

In [4]:
print(f"1 bar = {bar} Pa")
print(f"1 atm = {atm} Pa")
print(f"0 degree Celsius = {zero_Celsius} K")
print(f"The gas constant R is: {R} $J⋅K^(−1)⋅mol−1$")
print(f"standard acceleration of gravity: {g}")

1 bar = 100000.0 Pa
1 atm = 101325.0 Pa
0 degree Celsius = 273.15 K
The gas constant R is: 8.314462618 $J⋅K^(−1)⋅mol−1$
standard acceleration of gravity: 9.80665


In [5]:
# Define constant variables
STANDARD_TEMPERATURE = 15 + zero_Celsius  # standard temperature (15 degree Celsius in K)
STANDARD_PRESSURE = 1 * atm  # standard pressure in Pa
STANDARD_TEMPERATURE, STANDARD_PRESSURE

(288.15, 101325.0)

In [6]:
# Natural gas properties
RD = 0.554 
Z = 0.894
# Average molar mass of air
M_AIR = 28.97  # g/mol

### Define the functions to calculate flow rate

$$
Q = \pm\pi\sqrt{\frac{R}{16M_{air}}}\frac{T_{st}}{P_{st}}D^{2.5}\sqrt{\frac{|P_1^2-P_2^2-2g\sin\theta L\frac{P_{avg}^2dM_{air}}{Z_{avg}RT_{avg}}|}{LdTZ_{avg}f}}
$$

\begin{equation}
Q_{st} = sgn(P_1,P_2,E)\cdot C_1 \frac{T_{st}}{P_{st}}\left(\frac{|P_1^2-P_2^2-E|}{LdT_{avg}Z_{avg}}\right)^{0.5}\frac{D^{2.5}}{\sqrt{f}}\\\\
\end{equation}

\begin{equation}
sgn(P_1,P_2,E) = \left\{\begin{aligned}
 1&,~if ~P_1^2-P_2^2-E\ge0,\\
 -1&,~if~P_1^2-P_2^2-E<0.
\end{aligned} \right.
\end{equation}

\begin{equation}
E = C_2 d(H_2-H_1)\frac{P_{avg}^2}{T_{avg}Z_{avg}}
\end{equation}

In [7]:
# Determine the C1 and C2
C1 = pi * (R * 1000 / 16 / M_AIR)**0.5
C2 = 2 * g * M_AIR / (R * 1000)
C1, C2

(13.305546782440928, 0.06833842752145018)

In [8]:
# Function to determine the flow direction
def flow_direction(p1, p2, e):
    if (p1**2 - p2**2 - e) >= 0:
        return 1
    else:
        return -1

In [9]:
# Function to calculate the average temperature
def calculate_average_temperature(t1, t2):
    return (t1+t2) / 2

In [10]:
# Function to calculate the average pressure
def calculate_average_pressure(p1, p2):
    return 2/3 * ((p1+p2) - (p1*p2) / (p1 + p2))

In [11]:
# Function to calculate the height difference effect
def height_effect(h1, h2, p1, p2, t1, t2, d=RD, Z=Z):
    p_avg = calculate_average_pressure(p1, p2)
    t_avg = calculate_average_temperature(t1, t2)
    return C2 * d * (h2-h1) * p_avg**2 / (t_avg*Z)

\begin{equation}
Q_{ij}=\mathrm{sgn}(P_i,P_j,E_{ij})C_{ij}|P_i^2 - P_j^ 2-E_{ij}|^{0.5}
\end{equation}

In [13]:
# Create a helper function to calculate C_ij of a pipe which won't change during one iteration
def c_pipe(t1, t2, L, D, f=0.01, eta=0.85, Z=Z, d=RD):
    t_avg = calculate_average_temperature(t1, t2)
    return C1 * STANDARD_TEMPERATURE/STANDARD_PRESSURE * (D**2.5) * eta * (1 / (L*t_avg*f*d*Z))**0.5

In [14]:
def volumetric_flow_rate(p1, p2, t1, t2, h1, h2, L, D, d=RD, Z=Z, f=0.01, eta=0.85):
    # p_avg = calculate_average_pressure(p1, p2)
    t_avg = calculate_average_temperature(t1, t2)
    e = height_effect(h1, h2, p1, p2, t1, t2)
    f_direction = flow_direction(p1, p2, e)
    return f_direction * eta * C1 * STANDARD_TEMPERATURE/STANDARD_PRESSURE * (abs(p1**2 - p2**2 - e) / (L*d*t_avg*Z))**0.5 * D**2.5 / f**0.5

In [15]:
volumetric_flow_rate(p1=50*bar,
                     p2=48*bar,
                     t1=300,
                     t2=300,
                     h1=0,
                     h2=0,
                     L=100e3,
                     D=0.5)

20.650113399102267